In [211]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import datetime
from datetime import date
import seaborn as sns
from sklearn.preprocessing import *
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
%matplotlib inline

# 一、 Session Data

In [212]:
df_sessions=pd.read_csv('./Airbnb/sessions.csv')
df_sessions.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,jpmpj40cvk,lookup,NaN,NaN,Windows Desktop,957.0


### 1.把user_id改成id

In [213]:
df_sessions['id']=df_sessions['user_id']
df_sessions.drop(['user_id'],axis=1,inplace=True)

In [214]:
df_sessions.head(10)

,action,action_type,action_detail,device_type,secs_elapsed,id
0,lookup,NaN,NaN,Windows Desktop,319.0,d1mm9tcy42
1,search_results,click,view_search_results,Windows Desktop,67753.0,d1mm9tcy42
2,lookup,NaN,NaN,Windows Desktop,301.0,d1mm9tcy42
3,search_results,click,view_search_results,Windows Desktop,22141.0,d1mm9tcy42
4,lookup,NaN,NaN,Windows Desktop,957.0,jpmpj40cvk
5,search_results,click,view_search_results,Windows Desktop,965.0,jpmpj40cvk
6,search_results,click,view_search_results,Windows Desktop,1021.0,jpmpj40cvk
7,search_results,click,view_search_results,Windows Desktop,938.0,jpmpj40cvk
8,search_results,click,view_search_results,Windows Desktop,2702.0,jpmpj40cvk
9,ajax_refresh_subtotal,click,change_trip_characteristics,Windows Desktop,674.0,jpmpj40cvk


In [215]:
df_sessions.shape

(1048575, 6)

In [216]:
df_sessions.isnull().sum()

action             7873
action_type      143097
action_detail    143097
device_type           0
secs_elapsed      11067
id                 2797
dtype: int64

### 2.准备数据
>1.把空值统一为NAN

In [217]:
df_sessions.action=df_sessions.action.fillna('NAN')
df_sessions.action_type=df_sessions.action_type.fillna('NAN')
df_sessions.action_detail=df_sessions.action_detail.fillna('NAN')
df_sessions.device_type=df_sessions.device_type.fillna('NAN')
df_sessions.head()

,action,action_type,action_detail,device_type,secs_elapsed,id
0,lookup,NAN,NAN,Windows Desktop,319.0,d1mm9tcy42
1,search_results,click,view_search_results,Windows Desktop,67753.0,d1mm9tcy42
2,lookup,NAN,NAN,Windows Desktop,301.0,d1mm9tcy42
3,search_results,click,view_search_results,Windows Desktop,22141.0,d1mm9tcy42
4,lookup,NAN,NAN,Windows Desktop,957.0,jpmpj40cvk


> 2.把** action **里面数量较少的分成一类（设置一个阈值freq=100）

In [218]:
freq=100
act=dict(zip(*np.unique(df_sessions.action,return_counts=True)))
df_sessions.action=df_sessions.action.apply(lambda x: 'OTHER'if act[x]<freq else x)

In [219]:
df_sessions.action.value_counts().shape

(117,)

> 3. 利用 ** argsort ** 把action的统计排列整齐

In [220]:
f_act = df_sessions.action.value_counts().argsort()
f_act_detail = df_sessions.action_detail.value_counts().argsort()
f_act_type = df_sessions.action_type.value_counts().argsort()
f_dev_type = df_sessions.device_type.value_counts().argsort()

In [221]:
f_act.shape,f_act_detail.shape,f_act_type.shape,f_dev_type.shape

((117,), (108,), (10,), (14,))

> 4. 根据 id 把数据进行整合， 利用 groupby

In [222]:
dgr_sess=df_sessions.groupby(['id'])

In [223]:
len(dgr_sess)

15588

> 5. 处理所有的session数据结构

In [224]:
samples=[]
ln=len(dgr_sess)

In [225]:
for g in dgr_sess:
    gr=g[1]  #某一个id的所有信息
    l=[]     #设置一个空list存储处理后的特征（针对个g[0]这个id的）
    l.append(g[0])
    l.append(len(gr))
    
    sev=gr.secs_elapsed.fillna(0).values  # secs_elapsed所有的值
    
    c_act=[0]*len(f_act)
    for i,v in enumerate(gr.action.values):
        c_act[f_act[v]]+=1   #计算action同一种操作的次数
    m,n=np.unique(gr.action.values,return_counts=True)
    c_act+=[len(n),np.mean(n),np.std(n)] #统计出每个值发生的次数，均值，和标准差
    l=l+c_act
        
    #action_detail特征
    c_act_detail=[0]*len(f_act_detail)
    for i,v in enumerate(gr.action_detail.values):
        c_act_detail[f_act_detail[v]]+=1
    _,n_det=np.unique(gr.action_detail.values,return_counts=True)
    c_act_detail+=[len(n_det),np.mean(n_det),np.std(n_det)]
    l=l+c_act_detail
            
    #action_dtype特征
    l_act_type=[0]*len(f_act_type)
    c_act_type=[0]*len(f_act_type)
        
    for i,v in enumerate(gr.action_type.values):
        l_act_type[f_act_type[v]]+=sev[i]
        c_act_type[f_act_type[v]]+=1
    l_act_type=np.log(1+np.array(l_act_type)).tolist()
    _,n_typ=np.unique(gr.action_type.values,return_counts=True)
    c_act_type+=[len(n_typ),np.mean(n_typ),np.std(n_typ)]
    l=l+c_act_type+l_act_type
        
    #device_type features
    c_dev_type=[0]*len(f_dev_type)
    for i,v in enumerate(gr.device_type.values):
        c_dev_type[f_dev_type[v]]+=1
    _,n_dev=np.unique(gr.device_type.values,return_counts=True)
    c_dev_type+=[len(n_dev),np.mean(n_dev),np.std(n_dev)]
    l=l+c_dev_type
        
    #secs_elapsed features
    l_secs=[0]*5
    l_log=[0]*15
    if len(sev)>0:
        l_secs[0]=np.log(1+np.sum(sev))
        l_secs[1]=np.log(1+np.mean(sev))
        l_secs[2]=np.log(1+np.std(sev))
        l_secs[3]=np.log(1+np.median(sev))
        l_secs[4]=l_secs[0]/float(l[1])   #l[1]为gr的长度
            
        log_sev=np.log(1+sev).astype(int)
            
        l_log=np.bincount(log_sev,minlength=15).tolist()
    l=l+l_secs+l_log
        
    samples.append(l) 

In [226]:
samples=np.array(samples)
len(samples[-1])

293

> 6. 返回一个新的DataFrame

In [227]:
samples=np.array(samples)
samp_ar=samples[:,1:].astype(np.float16)
samp_id=samples[:,0]

In [228]:
col_names=[]
for i in range(len(samples[0])-1):
    col_names.append('c_'+str(i))

In [229]:
df_agg_sess=pd.DataFrame(samp_ar,columns=col_names)
df_agg_sess['id']=samp_id
df_agg_sess.index=df_agg_sess.id

In [230]:
df_agg_sess.head(10)

,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,...,c_283,c_284,c_285,c_286,c_287,c_288,c_289,c_290,c_291,id
id,,,,,,,,,,,,,,,,,,,,,
007gj7kqdk,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,2.0,1.0,2.0,0.0,1.0,0.0,0.0,007gj7kqdk
009a40t3dk,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,4.0,4.0,5.0,2.0,1.0,1.0,2.0,0.0,009a40t3dk
00allnceb8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00allnceb8
00e8bokexa,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00e8bokexa
00fhpdik5t,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,2.0,10.0,4.0,3.0,0.0,0.0,0.0,0.0,00fhpdik5t
00fn6wu77e,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,4.0,5.0,5.0,3.0,1.0,0.0,2.0,0.0,00fn6wu77e
00fyswof3k,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,4.0,4.0,11.0,16.0,8.0,3.0,1.0,0.0,00fyswof3k
00guzlz8b8,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,8.0,2.0,4.0,4.0,6.0,4.0,1.0,0.0,00guzlz8b8
00iapy7gb3,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,2.0,2.0,1.0,1.0,0.0,1.0,0.0,00iapy7gb3


## 完成数据清理

# 二、处理训练和测试集

In [174]:
train = pd.read_csv("train_users_2.csv")
test = pd.read_csv("test_users.csv")

In [175]:
train.shape,test.shape

((213451, 16), (62096, 15))

In [176]:
train.head(5)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [177]:
train_row=train.shape[0]  #计算出train的行数，便于之后对train和test数据进行分离操作
train_row

213451

> 1. 需要预测的项

In [178]:
labels=train['country_destination'].values
labels

array(['NDF', 'NDF', 'US', ..., 'NDF', 'NDF', 'NDF'], dtype=object)

> 2. 删除date_first_booking和train文件中的country_destination
    - 数据探索时我们发现date_first_booking在train和test文件中缺失值太多，故删除
    - 删除country_destination，用模型预测country_destination，再与已经存储country_destination的labels进行比较，从而判断模型优劣

In [179]:
train.drop(['country_destination','date_first_booking'],axis=1,inplace=True)
test.drop(['date_first_booking'],axis=1,inplace=True)
train.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,gxn3p5htnn,2010-06-28,20090319043255,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome
1,820tgsjxq7,2011-05-25,20090523174809,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome
2,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE
3,bjjt8pjhuk,2011-12-05,20091031060129,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox
4,87mebub9p4,2010-09-14,20091208061105,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome


> 3.合并 train 和 test ,方便特征提取

In [180]:
df=pd.concat([train,test],axis=0,ignore_index=True)
df.shape

(275547, 14)

>4. timestamp_first_active 转换为 datetime 类型

In [181]:
tfa=df.timestamp_first_active.astype(str).\
apply(lambda x: datetime.datetime.strptime(x,'%Y%m%d%H%M%S'))

In [182]:
tfa.shape

(275547,)

> 5.提取特征：年，月，日

In [183]:
df['tfa_year']=np.array([x.year for x in tfa])
df['tfa_month']=np.array([x.month for x in tfa])
df['tfa_day']=np.array([x.day for x in tfa])

In [184]:
df.columns

Index(['id', 'date_account_created', 'timestamp_first_active', 'gender', 'age',
       'signup_method', 'signup_flow', 'language', 'affiliate_channel',
       'affiliate_provider', 'first_affiliate_tracked', 'signup_app',
       'first_device_type', 'first_browser', 'tfa_year', 'tfa_month',
       'tfa_day'],
      dtype='object')

> 5.提取特征：weekday ,同时针对结果进行 **one hot encoding ** 编码

In [185]:
df['tfa_wd']=np.array([x.isoweekday() for x in tfa])
df_tfa_wd=pd.get_dummies(df.tfa_wd,prefix='tfa_wd')  

In [186]:
df=pd.concat((df,df_tfa_wd),axis=1)
df.drop(['tfa_wd'],axis=1,inplace=True)
df.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,...,tfa_year,tfa_month,tfa_day,tfa_wd_1,tfa_wd_2,tfa_wd_3,tfa_wd_4,tfa_wd_5,tfa_wd_6,tfa_wd_7
0,gxn3p5htnn,2010-06-28,20090319043255,-unknown-,NaN,facebook,0,en,direct,direct,...,2009,3,19,0,0,0,1,0,0,0
1,820tgsjxq7,2011-05-25,20090523174809,MALE,38.0,facebook,0,en,seo,google,...,2009,5,23,0,0,0,0,0,1,0
2,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,56.0,basic,3,en,direct,direct,...,2009,6,9,0,1,0,0,0,0,0
3,bjjt8pjhuk,2011-12-05,20091031060129,FEMALE,42.0,facebook,0,en,direct,direct,...,2009,10,31,0,0,0,0,0,1,0
4,87mebub9p4,2010-09-14,20091208061105,-unknown-,41.0,basic,0,en,direct,direct,...,2009,12,8,0,1,0,0,0,0,0


> 6.提取特征：季节

In [187]:
Y = 2000
seasons = [(0, (date(Y,  1,  1),  date(Y,  3, 20))),  #'winter'
           (1, (date(Y,  3, 21),  date(Y,  6, 20))),  #'spring'
           (2, (date(Y,  6, 21),  date(Y,  9, 22))),  #'summer'
           (3, (date(Y,  9, 23),  date(Y, 12, 20))),  #'autumn'
           (0, (date(Y, 12, 21),  date(Y, 12, 31)))]  #'winter'

def get_season(dt):
    dt=dt.date()
    dt=dt.replace(year=Y)
    return next(season for season,(start,end) in seasons if start <= dt <= end)

df['tfa_season']=np.array([get_season(x) for x in tfa])
df_tfa_season=pd.get_dummies(df.tfa_season,prefix='tfa_season')
df=pd.concat([df,df_tfa_season],axis=1)
df.drop(['tfa_season'],axis=1,inplace=True)

In [188]:
df.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,...,tfa_wd_2,tfa_wd_3,tfa_wd_4,tfa_wd_5,tfa_wd_6,tfa_wd_7,tfa_season_0,tfa_season_1,tfa_season_2,tfa_season_3
0,gxn3p5htnn,2010-06-28,20090319043255,-unknown-,NaN,facebook,0,en,direct,direct,...,0,0,1,0,0,0,1,0,0,0
1,820tgsjxq7,2011-05-25,20090523174809,MALE,38.0,facebook,0,en,seo,google,...,0,0,0,0,1,0,0,1,0,0
2,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,56.0,basic,3,en,direct,direct,...,1,0,0,0,0,0,0,1,0,0
3,bjjt8pjhuk,2011-12-05,20091031060129,FEMALE,42.0,facebook,0,en,direct,direct,...,0,0,0,0,1,0,0,0,0,1
4,87mebub9p4,2010-09-14,20091208061105,-unknown-,41.0,basic,0,en,direct,direct,...,1,0,0,0,0,0,0,0,0,1


> 7.date_account_created 转化为 datetime 格式

In [189]:
dac=pd.to_datetime(df.date_account_created)

> 8.提取特征：年，月，日, weekday, 季节

In [190]:
df['dac_year']=np.array([x.year for x in dac])
df['dac_month']=np.array([x.month for x in dac])
df['dac_day']=np.array([x.day for x in dac])

df['dac_wd']=np.array([x.isoweekday() for x in dac])
df_dac_wd=pd.get_dummies(df.dac_wd,prefix='dac_wd')
df=pd.concat([df,df_dac_wd],axis=1)
df.drop(['dac_wd'],axis=1,inplace=True)

df['dac_season']=np.array([get_season(x) for x in dac])
df_dac_season=pd.get_dummies(df.dac_season,prefix='dac_season')
df=pd.concat([df,df_dac_season],axis=1)
df.drop(['dac_season'],axis=1,inplace=True)

df.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,...,dac_wd_2,dac_wd_3,dac_wd_4,dac_wd_5,dac_wd_6,dac_wd_7,dac_season_0,dac_season_1,dac_season_2,dac_season_3
0,gxn3p5htnn,2010-06-28,20090319043255,-unknown-,NaN,facebook,0,en,direct,direct,...,0,0,0,0,0,0,0,0,1,0
1,820tgsjxq7,2011-05-25,20090523174809,MALE,38.0,facebook,0,en,seo,google,...,0,1,0,0,0,0,0,1,0,0
2,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,56.0,basic,3,en,direct,direct,...,1,0,0,0,0,0,0,0,0,1
3,bjjt8pjhuk,2011-12-05,20091031060129,FEMALE,42.0,facebook,0,en,direct,direct,...,0,0,0,0,0,0,0,0,0,1
4,87mebub9p4,2010-09-14,20091208061105,-unknown-,41.0,basic,0,en,direct,direct,...,1,0,0,0,0,0,0,0,1,0


> 9. 提取特征：date_account_created和timestamp_first_active之间的差值
        - 即用户在airbnb平台活跃到正式注册所花的时间
        - 分类为一天，一个月，一年，其他

In [191]:
dt_span=dac.subtract(tfa).dt.days 

In [192]:
dt_span.value_counts().head(10)

-1     275369
 0          7
 6          4
 5          4
 1          4
 2          3
 3          3
 4          3
 28         3
 94         2
dtype: int64

In [193]:
def get_span(dt):
    if dt==-1:return 'OneDay'
    elif (-1<dt<30):return 'OneMonth'
    elif (30<=dt<=365):return 'OneYear'
    else: return 'Other'

In [194]:
df['dt_span']=np.array([get_span(x) for x in dt_span])
df_dt_span=pd.get_dummies(df.dt_span,prefix='dt_span')
df=pd.concat([df,df_dt_span],axis=1)
df.drop(['dt_span'],axis=1,inplace=True)

df.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,...,dac_wd_6,dac_wd_7,dac_season_0,dac_season_1,dac_season_2,dac_season_3,dt_span_OneDay,dt_span_OneMonth,dt_span_OneYear,dt_span_Other
0,gxn3p5htnn,2010-06-28,20090319043255,-unknown-,NaN,facebook,0,en,direct,direct,...,0,0,0,0,1,0,0,0,0,1
1,820tgsjxq7,2011-05-25,20090523174809,MALE,38.0,facebook,0,en,seo,google,...,0,0,0,1,0,0,0,0,0,1
2,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,56.0,basic,3,en,direct,direct,...,0,0,0,0,0,1,0,0,0,1
3,bjjt8pjhuk,2011-12-05,20091031060129,FEMALE,42.0,facebook,0,en,direct,direct,...,0,0,0,0,0,1,0,0,0,1
4,87mebub9p4,2010-09-14,20091208061105,-unknown-,41.0,basic,0,en,direct,direct,...,0,0,0,0,1,0,0,0,1,0


> 10.删除原来的 timestamp_first_active，date_account_created

In [195]:
df.drop(['timestamp_first_active','date_account_created'],axis=1,inplace=True)
df.shape

(275547, 44)

> 11.处理Age
    - 在数据探索阶段，我们发现大部分数据是集中在（15，90）区间的，但有部分年龄分布在（1900，2000）区间，我们猜测用户是把出生日期误填为年龄，故进行预处理

In [196]:
av=df.age.values
av

array([nan, 38., 56., ..., nan, nan, 49.])

In [197]:
np.logical_and?
np.where?

In [198]:
av=np.where(np.logical_and(av<2000,av>1900),2014-av,av)
df['age']=av
age=df.age

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in less
  """Entry point for launching an IPython kernel.
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.


In [199]:
age.fillna(-1,inplace=True)

div=15

In [200]:
def get_age(age):
    # age is a float number  将连续型转换为离散型
    if age < 0:
        return 'NA' #表示是空值
    elif (age < div):
        return div #如果年龄小于15岁，那么返回15岁
    elif (age <= div * 2):
        return div*2 #如果年龄大于15小于等于30岁，则返回30岁
    elif (age <= div * 3):
        return div * 3
    elif (age <= div * 4):
        return div * 4
    elif (age <= div * 5):
        return div * 5
    elif (age <= 110):
        return div * 6
    else:
        return 'Unphysical' #非正常年龄  

In [201]:
df['age'] = np.array([get_age(x) for x in age])
df_age=pd.get_dummies(df.age,prefix='age')
df=pd.concat([df,df_age],axis=1)
df.drop(['age'],axis=1,inplace=True)
df.head()

,id,gender,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,...,dt_span_OneYear,dt_span_Other,age_15,age_30,age_45,age_60,age_75,age_90,age_NA,age_Unphysical
0,gxn3p5htnn,-unknown-,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,...,0,1,0,0,0,0,0,0,1,0
1,820tgsjxq7,MALE,facebook,0,en,seo,google,untracked,Web,Mac Desktop,...,0,1,0,0,1,0,0,0,0,0
2,4ft3gnwmtx,FEMALE,basic,3,en,direct,direct,untracked,Web,Windows Desktop,...,0,1,0,0,0,1,0,0,0,0
3,bjjt8pjhuk,FEMALE,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,...,0,1,0,0,1,0,0,0,0,0
4,87mebub9p4,-unknown-,basic,0,en,direct,direct,untracked,Web,Mac Desktop,...,1,0,0,0,1,0,0,0,0,0


In [203]:
df.shape

(275547, 51)

> 12.其他特征
    - 在数据探索时，我们发现剩余的特征lables都比较少，故不进一步进行特征提取，只进行one-hot-encoding处理

In [204]:
feat_toOHE = ['gender', 
             'signup_method', 
             'signup_flow', 
             'language', 
             'affiliate_channel', 
             'affiliate_provider', 
             'first_affiliate_tracked', 
             'signup_app', 
             'first_device_type', 
             'first_browser']

In [205]:
for f in feat_toOHE:
    df_ohe=pd.get_dummies(df[f],prefix=f,dummy_na=True)
    df=pd.concat((df,df_ohe),axis=1)
    df.drop([f],axis=1,inplace=True)

In [206]:
df.shape

(275547, 204)

In [207]:
df.head()

,id,tfa_year,tfa_month,tfa_day,tfa_wd_1,tfa_wd_2,tfa_wd_3,tfa_wd_4,tfa_wd_5,tfa_wd_6,...,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_UC Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser,first_browser_nan
0,gxn3p5htnn,2009,3,19,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,820tgsjxq7,2009,5,23,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,4ft3gnwmtx,2009,6,9,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,bjjt8pjhuk,2009,10,31,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,87mebub9p4,2009,12,8,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [236]:
df_all=pd.merge(df,df_agg_sess,how='left')
df_all.drop(['id'],axis=1,inplace=True)
df_all=df_all.fillna(-2)

#加了一列，表示每一行总共有多少空值，这也作为一个特征


C:\Users\Administrator\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2963: FutureWarning: 'id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [238]:
df_all['all_null']=np.array([sum(r<0) for r in df_all.values])

In [239]:
df_all.head()

,tfa_year,tfa_month,tfa_day,tfa_wd_1,tfa_wd_2,tfa_wd_3,tfa_wd_4,tfa_wd_5,tfa_wd_6,tfa_wd_7,...,c_283,c_284,c_285,c_286,c_287,c_288,c_289,c_290,c_291,all_null
0,2009,3,19,0,0,0,1,0,0,0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,292
1,2009,5,23,0,0,0,0,0,1,0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,292
2,2009,6,9,0,1,0,0,0,0,0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,292
3,2009,10,31,0,0,0,0,0,1,0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,292
4,2009,12,8,0,1,0,0,0,0,0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,292


In [240]:
df_all.shape

(275547, 496)

> 13.将train和test数据进行分离操作

In [242]:
Xtrain=df_all.iloc[:train_row,:]
Xtest=df_all.iloc[train_row:,:]

In [244]:
Xtrain.shape,Xtest.shape

((213451, 496), (62096, 496))

> 14.将提取的特征生成 csv 文件

In [247]:
Xtrain.to_csv('Airbnb_xtrain_jim.csv')
Xtest.to_csv('Airbnb_xtest_jim.csv')

In [248]:
labels.tofile('Airbnb_ytrain_jim.csv',sep='\n',format='%s')